## 搶票範例1: 自動驗證
- ### 目標網站: https://kham.com.tw/application/UTK02/UTK0201_00.aspx?PRODUCT_ID=N0L1AZAH   (寬宏售票)

- ### 載入套件

    自行下載安裝PIL及pytesseract以及tesseract套件

In [1]:
from selenium import webdriver
from PIL import Image                                # 用來處理圖片
import pytesseract                                   # Google開源庫，用來做文字辨識 (需安裝 tesseract)  
import re                                            # 正規表達式模組 (非必要式驗證碼情況而定)

- ### 開啟瀏覽器與網站

In [ ]:
driver = webdriver.Chrome()
driver.get('https://kham.com.tw/application/UTK02/UTK0201_00.aspx?PRODUCT_ID=N0L1AZAH')

- ### 開啟全螢幕

    因為等等要擷取螢幕上的驗證碼，不開全螢幕的話比例會跑掉

In [ ]:
driver.maximize_window()

- ### 依序點擊想要的訂票資訊

    補充說明:
    1. 實際情況中，應可先觀察其他售票活動的流程，搭配預先知道的資訊來做爬蟲
    2. 舉例來說，不同活動的標籤/id可能不同，因此多使用預先知道的資訊如:活動時間/地點，做為執行爬蟲的依據

- ### 選擇場次

    因寬宏的網頁結構教不嚴謹，重複的tag很多，缺乏屬性參考，所以需要用較複雜的表示法定位

In [ ]:
# 選擇場次表中，align屬性是left的列，因為除了表頭是center外，其他都是left，也就是選取所有的活動場次
events = driver.find_elements_by_css_selector('#ctl00_ContentPlaceHolder1_DataGrid tr[align=left]')

# 迭代所有場次，場次中第一個資訊是日期，這裡假定已經預先看好要買的場次日期，所以直接對每個場次做比對
# 找到目標場次後，點擊立即訂購 (一列的每個資訊中，只有最後一個立即訂購的align是right)
for event in events:
    if '2020/12/12' in event.find_element_by_css_selector('td[align=left]').text:
        event.find_element_by_css_selector('td[align=right]').click()
        break

- ### 選取位置

    這裡有的系統會有自行選位或是電腦配位的選擇，選擇電腦配位可以節省時間

In [ ]:
# 選擇電腦配位
driver.find_element_by_id('ctl00_ContentPlaceHolder1_BUY_TYPE_2').click()

# 選擇區域
# 這裡直接用id選，但實際操作可以根據個人需求，如價格區段，另外寫邏輯
driver.find_element_by_id('N0L1AZBJ').click()

- ### 選擇張數

In [ ]:
driver.find_element_by_id('ctl00_ContentPlaceHolder1_DataGrid_ctl02_AMOUNT').send_keys('2')

- ### 登入

    注意到目前為止我們還沒實際登入系統，寬宏的系統可以在這個步驟再登入

In [ ]:
driver.find_element_by_id('ctl00_ContentPlaceHolder1_LOGIN_ID').send_keys('YOUR_ACCOUNT')
driver.find_element_by_id('ctl00_ContentPlaceHolder1_LOGIN_PWD').send_keys('YOUR_PASSWD')

- ### 驗證碼處理

    為了處理驗證碼，我們需要執行以下步驟:
    
    1. 螢幕截圖(全螢幕)，下載到本地
    2. 開啟截圖，再擷取驗整碼圖片
    3. 辨識驗證碼
    4. 將辨識結果輸入網頁空格

- ### 螢幕截圖

In [ ]:
driver.save_screenshot('test_img.png')

- ### 擷取驗證碼圖片

In [ ]:
# 取得網頁上的驗證碼tag
ver_code_tag = driver.find_element_by_id('chk_pic')

# 取得tag座標
loc = ver_code_tag.location

# 取得圖片長寬
width = ver_code_tag.size['width']
height = ver_code_tag.size['height']

# 讀取剛剛的全螢幕截圖
img = Image.open('test_img.png')

# 用座標+長寬來界定驗證碼圖片的區塊
box = (loc['x'], loc['y'], loc['x']+width, loc['y']+height)

# 取得驗證碼圖片
ver_img = img.crop(box)

- ### 辨識驗證碼

    這裡要進行圖片去噪的動作，為了方便使用，將去噪過程寫成函式

In [ ]:
def denoise(img):
    grey = img.convert('L')                                # 灰階轉換
    text = pytesseract.image_to_string(grey)               # 使用pytesseract辨識
    res = re.sub('\W', '', text)                           # 去除非字元或數字，即去除符號與空格

    return res


ver_code = denoise(ver_img)
print('Verification Code: {}'.format(ver_code))

- ### 將結果輸入驗證碼空格

In [ ]:
driver.find_element_by_id('ctl00_ContentPlaceHolder1_CHK').send_keys(ver_code)

- ### 點擊加入購物車

In [ ]:
driver.find_element_by_id('ctl00_ContentPlaceHolder1_AddShopingCart').click()

- ### 結尾

    若程式執行至此沒有錯誤，則表示順利搶到票，可以慢慢填付款資訊送出
    
    最後別忘記關閉連線

In [ ]:
driver.quit()